In [1]:
import tensorflow as tf 
import numpy as np

In [20]:
def conv_block(x, filters, block_num, conv_num, strides=(1,1)):
    name = 'block{}_conv{}_'.format(block_num, conv_num)     
    # conv-BN-relu 
    x = tf.keras.layers.Conv2D(filters, (3,3), strides=strides, use_bias=False, name=name)(x) 
    x = tf.keras.layers.BatchNormalization(name=name+'bn')(x)   
    x = tf.keras.layers.Activation('relu', name=name+'act')(x)     
    return x

In [3]:
def separable_conv_block(x, filters, block_num, conv_num, pre_activation=None): 
    name = 'block{}_sepconv{}_'.format(block_num, conv_num)     
    if pre_activation is True:        
        x = tf.keras.layers.Activation('relu', name=name+'act')(x)      
    # (relu)-sepconv-BN-(relu)  
    x = tf.keras.layers.SeparableConv2D(filters, (3,3), padding='same', use_bias=False, name=name)(x)   
    x = tf.keras.layers.BatchNormalization(name=name+'bn')(x)  
    
    if pre_activation is False:        
        x = tf.keras.layers.Activation('relu', name=name+'act')(x)      
    return x

In [4]:
def middle_flow_block(x, filters, block_num):     
    # middle flow    
    residual = x     
    x = separable_conv_block(x, filters, block_num=block_num, conv_num='1', pre_activation=True)  
    x = separable_conv_block(x, filters, block_num=block_num, conv_num='2', pre_activation=True)   
    x = separable_conv_block(x, filters, block_num=block_num, conv_num='3', pre_activation=True)      
    return tf.keras.layers.add([x, residual])

In [5]:
def xception_block(x, filters, block_num, pre_activation=True):   
    block = 'block{}_'.format(block_num)    
    filter_conv1, filter_conv2 = filters     
    # residual conv branch     
    residual = tf.keras.layers.Conv2D(filter_conv2, (1, 1), strides=(2, 2),padding='same', use_bias=False)(x)   
    residual = tf.keras.layers.BatchNormalization()(residual)     
    # separable conv block  
    x = separable_conv_block(x, filter_conv1, block_num=block_num, conv_num='1', pre_activation=pre_activation)  
    x = separable_conv_block(x, filter_conv2, block_num=block_num, conv_num='2', pre_activation=True)     
    # downsampling and merging   
    x = tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same', name=block+'pool')(x)     
    return tf.keras.layers.add([x, residual])

In [6]:
def Xception(input_shape=(299,299,3), classes=1000):     
    """Instantiates the Xception architecture.
    """      
    img_input = tf.keras.layers.Input(shape=input_shape)      
    #===========ENTRY FLOW==============    
    #Block 1    
    x = conv_block(img_input, 32, block_num='1', conv_num='1', strides=(2,2))     
    x = conv_block(x, 64, block_num='1', conv_num='2')     
    #Block 2
    x = xception_block(x, (128, 128), '2', pre_activation=False)      
    #Block 3     
    x = xception_block(x, (256, 256), '3')      
    #Block 4     
    x = xception_block(x, (728, 728), '4')       
    #===========MIDDLE FLOW=============== 
    for i in range(8):         
        block_num = str(5+i)         
        x = middle_flow_block(x, 728, block_num)      
    #========EXIT FLOW============    
    #Block 13     
    x = xception_block(x, (728, 1024), '13') 
    # second conv is different  
    # Block 14    
    x = separable_conv_block(x, 1536, block_num='14', conv_num='1', pre_activation=False)
    x = separable_conv_block(x, 2048, block_num='14', conv_num='2', pre_activation=False)      
    # logistic regression    
    x = tf.keras.layers.GlobalAveragePooling2D(name='avg_pool')(x)    
    x = tf.keras.layers.Dense(classes, activation='softmax', name='predictions')(x)     
    # Create model.
    model = tf.keras.Model(inputs=img_input, outputs=x, name='xception')    
    return model

In [21]:
xception = Xception() 
xception.summary()

Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1_ (Conv2D)         (None, 149, 149, 32  864         ['input_12[0][0]']               
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 149, 149, 32  128        ['block1_conv1_[0][0]']          
 ation)                         )                                                          

In [19]:
base = tf.keras.applications.xception.Xception(weights=None, include_top=False, input_shape=(299, 299, 3))
x = base.output
x = tf.keras.layers.GlobalAveragePooling2D()(x) 
# add a fully-connected layer    
x = tf.keras.layers.Dense(1000, activation='relu')(x)    
# putput layer    
# predictions = tf.keras.layers.Dense(session.training_dataset_info['number_of_labels'], activation='softmax')(x) 
# predictions = tf.keras.layers.Dense(1000, activation='softmax', name='predictions')(x) 
# model   
model = tf.keras.Model(inputs=base.input, outputs=x)
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 149, 149, 32  864         ['input_11[0][0]']               
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 149, 149, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                           